### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

 you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [1]:
### Open AI API Key and Open Source models--Llama3,Gemma2,mistral--Groq

import os
from dotenv import load_dotenv
load_dotenv()

import openai
openai.api_key=os.getenv("OPEN_API_KEY")

groq_api_key=os.getenv("GROQ_API_KEY")


In [2]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000207234F2620>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000207234F0190>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage,SystemMessage

messages=[
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello How are you?")
]

result=model.invoke(messages)

In [4]:
result

AIMessage(content="**French:**\n\n* **Bonjour**\n* **Comment allez-vous ?** (formal)\n* **Comment vas-tu ?** (informal)\n\n\nLet me know if you'd like to translate anything else!\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 21, 'total_tokens': 71, 'completion_time': 0.090909091, 'prompt_time': 0.001323259, 'queue_time': 0.24740711, 'total_time': 0.09223235}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--0ad1215b-00b6-4207-a848-98ab658c0910-0', usage_metadata={'input_tokens': 21, 'output_tokens': 50, 'total_tokens': 71})

In [5]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(result)

"**French:**\n\n* **Bonjour**\n* **Comment allez-vous ?** (formal)\n* **Comment vas-tu ?** (informal)\n\n\nLet me know if you'd like to translate anything else!\n"

In [6]:
### Using LCEL- chain the components
chain=model|parser
chain.invoke(messages)

'Here are a couple of ways to translate "Hello, how are you?" to French:\n\n* **Bonjour, comment allez-vous ?** (Formal)\n* **Salut, comment vas-tu ?** (Informal)\n\n\nLet me know if you\'d like to see other variations!\n'

In [7]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template="Trnaslate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)



In [8]:
result=prompt.invoke({"language":"French","text":"Hello"})

In [9]:
result.to_messages()

[SystemMessage(content='Trnaslate the following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [10]:
##Chaining together components with LCEL
chain=prompt|model|parser
chain.invoke({"language":"French","text":"Hello"})

'Bonjour \n'